# 04 network graph and metrics
---

**Author:** Chen Weng

**Date:** August 20, 2024



---

# Load packages & Functions

In [2]:
library(data.table)
library(dplyr)
library(redeemR)
library(ggplot2)
library(gridExtra)
library(ggsci)
library(ggvenn)
library(igraph)
get_dist<-function(Cts.Mtx.bi){
    V.weight<-data.frame(weight=1-CellPCT$muRate)
    V.weight$Variants<-paste("Variants",gsub("_","",CellPCT$Variant),sep="")
    weight<-data.frame(Variants=colnames(Cts.Mtx.bi)) %>% merge(.,V.weight,by="Variants",all.x = T,sort = F) %>% .$weight
    weight[is.na(weight)]<-1
    d.w_jaccard<-quick_w_jaccard(Cts.Mtx.bi,w=weight)
    return(d.w_jaccard)
}
compute_largest_subgraph <- function(mtx,n=1){
    adj.mtx <- (mtx %*% t(mtx)) >= n
    diag(adj.mtx)<-0
    mc <- graph_from_adjacency_matrix(adj.mtx) %>% components(mode = "weak") 
    return(max(mc$csize)/nrow(mtx))

}

get_adj_graph <- function(mtx,n=1){
    adj.mtx <- (mtx %*% t(mtx)) >= n
    diag(adj.mtx)<-0
    return(graph_from_adjacency_matrix(adj.mtx, mode = "undirected"))
}
Compute_redeemR_adj_metrics <- function(redeemR_input, name){
    redeemR_obj <- Make_matrix(redeemR_input$redeemR, onlyhetero=T)
    adjacency_graph <- get_adj_graph(redeemR_obj@Cts.Mtx.bi)
    largest_subgraph <- compute_largest_subgraph(redeemR_obj@Cts.Mtx.bi)
    avg_degree <- mean(degree(adjacency_graph))
    graph_diameter <- diameter(adjacency_graph, directed=FALSE)
    avg_path_length <- mean_distance(adjacency_graph, directed=FALSE)
    Transitivity <- transitivity(adjacency_graph, type="global")
    print(name)
    print(paste("largest_subgraph", largest_subgraph))
    
    print(paste("Average degree", avg_degree))
    print(paste("Diameter", graph_diameter))
    print(paste("Average Path Length", avg_path_length))
    print(paste("Clustering Coefficient (Transitivity)", Transitivity))
    return(list(metric=c(name, avg_degree, graph_diameter, avg_path_length, Transitivity), graph = adjacency_graph))
}
# Plot_redeemR_graph <- function(redeemR_input, name){
#     redeemR_obj <- Make_matrix(redeemR_input$redeemR, onlyhetero=T)
#     Y1_BMMC.filter1.adjacency_graph <- get_adj_graph(Y1_BMMC.filter1$redeemR@Cts.Mtx.bi)
# }


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message:
“replacing previous import ‘ape::where’ by ‘dplyr::where’ when loading ‘redeemR’”
Warning message:
“replacing previous import ‘ape::keep.tip’ by ‘tidytree::keep.tip’ when loading ‘redeemR’”
Warning message:
“replacing previous import ‘ape::drop.tip’ by ‘tidytree::drop.tip’ when loading ‘redeemR’”

Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine


Loading required package: grid


Attaching package: ‘igraph’


The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    u

Prepare preprocessed filter1 and filter2 redeem objject and QC using ```../../runscript/prepare_redeemr_trim_test.r and prepare_redeemr.r``` to get prepare_redeem_*.rds

In [3]:

Y1_BMMC.filter1 <- readRDS("Path/prepare_redeemr_v1.young1_BMMC.rds")
Y1_BMMC.filter2 <- readRDS("Path/prepare_redeemr_trim5_binom.young1_BMMC.rds")

Y1_HSC.filter1 <- readRDS("Path/output/prepare_redeemr_v1.young1_HSC.rds")
Y1_HSC.filter2 <- readRDS("Path/output/prepare_redeemr_trim5_binom.young1_HSC.rds")

Y1_HPC.filter1 <- readRDS("Path/prepare_redeemr_v1.young1_HPC.rds")
Y1_HPC.filter2 <- readRDS("Path/prepare_redeemr_trim5_binom.young1_HPC.rds")

Y2_BMMC.filter1 <- readRDS("Path/prepare_redeemr_v1.young2_BMMC.rds")
Y2_BMMC.filter2 <- readRDS("Path/prepare_redeemr_trim5_binom.young2_BMMC.rds")

Y2_HSC.filter1 <- readRDS("Path/prepare_redeemr_v1.young2_HSC.rds")
Y2_HSC.filter2 <- readRDS("Path/prepare_redeemr_trim5_binom.young2_HSC.rds")

Y2_HPC.filter1 <- readRDS("Path/prepare_redeemr_v1.young2_HPC.rds")
Y2_HPC.filter2 <- readRDS("Path/prepare_redeemr_trim5_binom.young2_HPC.rds")

O1_HSPC.filter1 <- readRDS("Path/prepare_redeemr_v1.Old1_HSPC.rds")
O1_HSPC.filter2 <- readRDS("Path/prepare_redeemr_trim5_binom.Old1_HSPC.rds")

O1_BMMC.filter1 <- readRDS("Path/prepare_redeemr_v1.Old1_BMMC.rds")
O1_BMMC.filter2 <- readRDS("Path/prepare_redeemr_trim5_binom.Old1_BMMC.rds")

O2_HSPC.filter1 <- readRDS("Path/prepare_redeemr_v1.Old2_HSPC.rds")
O2_HSPC.filter2 <- readRDS("Path/prepare_redeemr_trim5_binom.Old2_HSPC.rds")

O2_BMMC.filter1 <- readRDS("Path/prepare_redeemr_v1.Old2_BMMC.rds")
O2_BMMC.filter2 <- readRDS("Path/prepare_redeemr_trim5_binom.Old2_BMMC.rds")

data(CellPCT)
V.weight<-data.frame(weight=1-CellPCT$muRate)
V.weight$Variants<-paste("Variants",gsub("_","",CellPCT$Variant),sep="")


## Calculate graph metrics from redeem R project for each sample

In [ ]:
Y1_BMMC.filter1.metric <- Compute_redeemR_adj_metrics(Y1_BMMC.filter1, "Y1_BMMC.filter1")
Y1_BMMC.filter2.metric <- Compute_redeemR_adj_metrics(Y1_BMMC.filter2, "Y1_BMMC.filter2")
Y1_HSC.filter1.metric <- Compute_redeemR_adj_metrics(Y1_HSC.filter1, "Y1_HSC.filter1")
Y1_HSC.filter2.metric <- Compute_redeemR_adj_metrics(Y1_HSC.filter2, "Y1_HSC.filter2")
Y1_HPC.filter1.metric <- Compute_redeemR_adj_metrics(Y1_HPC.filter1, "Y1_HPC.filter1")
Y1_HPC.filter2.metric <- Compute_redeemR_adj_metrics(Y1_HPC.filter2, "Y1_HPC.filter2")

Y2_BMMC.filter1.metric <- Compute_redeemR_adj_metrics(Y2_BMMC.filter1, "Y2_BMMC.filter1")
Y2_BMMC.filter2.metric <- Compute_redeemR_adj_metrics(Y2_BMMC.filter2, "Y2_BMMC.filter2")
Y2_HSC.filter1.metric <- Compute_redeemR_adj_metrics(Y2_HSC.filter1, "Y2_HSC.filter1")
Y2_HSC.filter2.metric <- Compute_redeemR_adj_metrics(Y2_HSC.filter2, "Y2_HSC.filter2")
Y2_HPC.filter1.metric <- Compute_redeemR_adj_metrics(Y2_HPC.filter1, "Y2_HPC.filter1")
Y2_HPC.filter2.metric <- Compute_redeemR_adj_metrics(Y2_HPC.filter2, "Y2_HPC.filter2")

O1_HSPC.filter1.metric <- Compute_redeemR_adj_metrics(O1_HSPC.filter1, "O1_HSPC.filter1")
O1_HSPC.filter2.metric <- Compute_redeemR_adj_metrics(O1_HSPC.filter2, "O1_HSPC.filter2")
O1_BMMC.filter1.metric <- Compute_redeemR_adj_metrics(O1_BMMC.filter1, "O1_BMMC.filter1")
O1_BMMC.filter2.metric <- Compute_redeemR_adj_metrics(O1_BMMC.filter2, "O1_BMMC.filter2")

O2_HSPC.filter1.metric <- Compute_redeemR_adj_metrics(O2_HSPC.filter1, "O2_HSPC.filter1")
O2_HSPC.filter2.metric <- Compute_redeemR_adj_metrics(O2_HSPC.filter2, "O2_HSPC.filter2")
O2_BMMC.filter1.metric <- Compute_redeemR_adj_metrics(O2_BMMC.filter1, "O2_BMMC.filter1")
O2_BMMC.filter2.metric <- Compute_redeemR_adj_metrics(O2_BMMC.filter2, "O2_BMMC.filter2")


Loading required package: Matrix.utils

Loading required package: Matrix

Only heteroplasmic mutations are used

@Cts.Mtx and @Cts.Mtx.bi are added



[1] "Y1_BMMC.filter1"
[1] "largest_subgraph 1"
[1] "Average degree 233.508514986376"
[1] "Diameter 5"
[1] "Average Path Length 2.11409797475886"
[1] "Clustering Coefficient (Transitivity) 0.347531899346927"


Only heteroplasmic mutations are used

@Cts.Mtx and @Cts.Mtx.bi are added



[1] "Y1_BMMC.filter2"
[1] "largest_subgraph 0.997324670361169"
[1] "Average degree 34.1995031530671"
[1] "Diameter 7"
[1] "Average Path Length 3.07340173574755"
[1] "Clustering Coefficient (Transitivity) 0.56506898497628"


Only heteroplasmic mutations are used

@Cts.Mtx and @Cts.Mtx.bi are added



...

Only heteroplasmic mutations are used

@Cts.Mtx and @Cts.Mtx.bi are added



[1] "O1_BMMC.filter2"
[1] "largest_subgraph 0.998725573491929"
[1] "Average degree 70.5729255168507"
[1] "Diameter 6"
[1] "Average Path Length 2.71104308500534"
[1] "Clustering Coefficient (Transitivity) 0.524413080982715"


Only heteroplasmic mutations are used

@Cts.Mtx and @Cts.Mtx.bi are added



[1] "O2_HSPC.filter1"
[1] "largest_subgraph 1"
[1] "Average degree 9396.42979656961"
[1] "Diameter 3"
[1] "Average Path Length 1.06296945196513"
[1] "Clustering Coefficient (Transitivity) 0.991242673630241"


Only heteroplasmic mutations are used

@Cts.Mtx and @Cts.Mtx.bi are added



In [ ]:
png("../plots/Y1_BMMC.filter1.graph.png")
plot(Y1_BMMC.filter1.metric$graph, layout=layout_with_kk, vertex.size = 1, vertex.label = NA, edge.label = NA)
dev.off()
png("../plots/Y1_BMMC.filter2.graph.png")
plot(Y1_BMMC.filter2.metric$graph, layout=layout_with_kk, vertex.size = 1, vertex.label = NA, edge.label = NA)
dev.off()
png("../plots/Y2_BMMC.filter1.graph.png")
plot(Y2_BMMC.filter1.metric$graph, layout=layout_with_kk, vertex.size = 1, vertex.label = NA, edge.label = NA)
dev.off()
png("../plots/Y2_BMMC.filter2.graph.png")
plot(Y2_BMMC.filter2.metric$graph, layout=layout_with_kk, vertex.size = 1, vertex.label = NA, edge.label = NA)
dev.off()

png("../plots/Y1_HSC.filter1.graph.png")
plot(Y1_HSC.filter1.metric$graph, layout=layout_with_kk, vertex.size = 1, vertex.label = NA, edge.label = NA)
dev.off()
png("../plots/Y1_HSC.filter2.graph.png")
plot(Y1_HSC.filter2.metric$graph, layout=layout_with_kk, vertex.size = 1, vertex.label = NA, edge.label = NA)
dev.off()
png("../plots/Y2_HSC.filter1.graph.png")
plot(Y2_HSC.filter1.metric$graph, layout=layout_with_kk, vertex.size = 1, vertex.label = NA, edge.label = NA)
dev.off()
png("../plots/Y2_HSC.filter2.graph.png")
plot(Y2_HSC.filter2.metric$graph, layout=layout_with_kk, vertex.size = 1, vertex.label = NA, edge.label = NA)
dev.off()

png("../plots/Y1_HPC.filter1.graph.png")
plot(Y1_HPC.filter1.metric$graph, layout=layout_with_kk, vertex.size = 1, vertex.label = NA, edge.label = NA)
dev.off()
png("../plots/Y1_HPC.filter2.graph.png")
plot(Y1_HPC.filter2.metric$graph, layout=layout_with_kk, vertex.size = 1, vertex.label = NA, edge.label = NA)
dev.off()
png("../plots/Y2_HPC.filter1.graph.png")
plot(Y2_HPC.filter1.metric$graph, layout=layout_with_kk, vertex.size = 1, vertex.label = NA, edge.label = NA)
dev.off()
png("../plots/Y2_HPC.filter2.graph.png")
plot(Y2_HPC.filter2.metric$graph, layout=layout_with_kk, vertex.size = 1, vertex.label = NA, edge.label = NA)
dev.off()

png("../plots/O1_HSPC.filter1.graph.png")
plot(O1_HSPC.filter1.metric$graph, layout=layout_with_kk, vertex.size = 1, vertex.label = NA, edge.label = NA)
dev.off()
png("../plots/O1_HSPC.filter2.graph.png")
plot(O1_HSPC.filter2.metric$graph, layout=layout_with_kk, vertex.size = 1, vertex.label = NA, edge.label = NA)
dev.off()
png("../plots/O2_HSPC.filter1.graph.png")
plot(O2_HSPC.filter1.metric$graph, layout=layout_with_kk, vertex.size = 1, vertex.label = NA, edge.label = NA)
dev.off()
png("../plots/O2_HSPC.filter2.graph.png")
plot(O2_HSPC.filter2.metric$graph, layout=layout_with_kk, vertex.size = 1, vertex.label = NA, edge.label = NA)
dev.off()

png("../plots/O1_BMMC.filter1.graph.png")
plot(O1_BMMC.filter1.metric$graph, layout=layout_with_kk, vertex.size = 1, vertex.label = NA, edge.label = NA)
dev.off()
png("../plots/O1_BMMC.filter2.graph.png")
plot(O1_BMMC.filter2.metric$graph, layout=layout_with_kk, vertex.size = 1, vertex.label = NA, edge.label = NA)
dev.off()
png("../plots/O2_BMMC.filter1.graph.png")
plot(O2_BMMC.filter1.metric$graph, layout=layout_with_kk, vertex.size = 1, vertex.label = NA, edge.label = NA)
dev.off()
png("../plots/O2_BMMC.filter2.graph.png")
plot(O2_BMMC.filter2.metric$graph, layout=layout_with_kk, vertex.size = 1, vertex.label = NA, edge.label = NA)
dev.off()